<a href="https://colab.research.google.com/github/randywreed/lex_article_review/blob/main/Tuesday_data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade gspread

     |████████████████████████████████| 114 kB 8.3 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 45.0 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 7.1 MB/s  eta 0:00:01
     |████████████████████████████████| 77 kB 9.2 MB/s  eta 0:00:01
     |████████████████████████████████| 147 kB 74.7 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
!pip uninstall -y nlpaug
!pip install numpy>=1.16.5 git+https://github.com/makcedward/nlpaug.git


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
import numpy
numpy.__version__

'1.19.5'

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
#from google.colab import auth
#auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('left_out_creation_main').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
#print(rows)

# Convert to a DataFrame and render.
import pandas as pd
tdf=pd.DataFrame.from_records(rows)
headers=tdf.iloc[0]
df=pd.DataFrame(tdf.values[1:],columns=headers)

ModuleNotFoundError: No module named 'oauth2client'

In [5]:
import pandas as pd
df=pd.read_csv('/spell/leftout/left_out_creation_main.csv')

In [4]:
df.columns

Index(['Unnamed: 0', 'What is left out(%)', 'Creation Content',
       'Creation Content_last', 'Left_out_flag'],
      dtype='object')

In [5]:
df=df[df['Creation Content_last'].notna()]
df.head()

,Unnamed: 0,What is left out(%),Creation Content,Creation Content_last,Left_out_flag
0,0,100.000000,In his article titled “Some Real News About Fa...,that affirms their personal opinion. Graham en...,True
1,14,100.000000,News Source (citation): “92% Of Indian Employe...,really portray two sides of the topic discusse...,True
3,28,100.000000,"The article ""The Story of the Birth of Jesus"" ...",first part of the summary starts off in the Go...,True
4,43,100.000000,The article in the Christian Post called “Half...,not be allowed to pick and choose what they co...,True
5,23,97.331835,"In the article, it is mentioned that there are...",how far can this go? Is it possible for these ...,True


In [2]:
import os
os.environ["MODEL_DIR"] = '/spell/content/'
model_dir='/spell/content'

In [4]:
len(df)

NameError: name 'df' is not defined

In [3]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [9]:
df.head()

,Unnamed: 0,What is left out(%),Creation Content,Creation Content_last,Left_out_flag
0,0,100.000000,In his article titled “Some Real News About Fa...,that affirms their personal opinion. Graham en...,True
1,14,100.000000,News Source (citation): “92% Of Indian Employe...,really portray two sides of the topic discusse...,True
3,28,100.000000,"The article ""The Story of the Birth of Jesus"" ...",first part of the summary starts off in the Go...,True
4,43,100.000000,The article in the Christian Post called “Half...,not be allowed to pick and choose what they co...,True
5,23,97.331835,"In the article, it is mentioned that there are...",how far can this go? Is it possible for these ...,True


In [10]:
df.shape

(60, 5)

In [4]:
#split the paragraph in to sentences.
def split_para(text):
  from nltk import tokenize
  out=tokenize.sent_tokenize(text)
  return out

In [5]:
def keyboard_aug(text):
  aug = nac.KeyboardAug()
  augmented_text = aug.augment(text)
  return augmented_text

In [6]:
def spelling_aug(text):
  aug=naw.SpellingAug()
  augmented_text= aug.augment(text, n=1)
  return augmented_text

In [7]:
def word2vec_aug(text):
  aug = naw.WordEmbsAug(
    model_type='word2vec', model_path='/spell/leftout/GoogleNews-vectors-negative300.bin',
    action="substitute")
  augmented_text = aug.augment(text)
  return augmented_text

In [8]:
def bert_sub_aug(text):
    aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="substitute")
    augmented_text = aug.augment(text)
    return augmented_text

In [9]:
def bert_ins_aug(text):
    aug = naw.ContextualWordEmbsAug(
    model_path='distilbert-base-uncased', action="insert")
    augmented_text = aug.augment(text)
    return augmented_text

In [52]:
def xlnet_sub_aug(text):
    aug=naw.ContextualWordEmbsAug(
    model_path='xlnet-base-cased',action="substitute")
    augmented_text=aug.augment(text)
    return augmented_text

In [44]:
def run_augmentation(func,newaugs,df):
    for idx,row in df.iterrows():
        #text=row['Creation Content_last']
        text=str(row['text'])
        #print(type(text))
        #flag=row['Left_out_flag']
        flag=row['label']
        sents=split_para(text)
        s=len(sents)
        #print('num of sent {}'.format(s))
        cnt=0
        #change one sentence each time.
        for i in range(s):
            augmented_text=func(sents[i])
            new_text=''
            for k in range(s):
                if k==i:
                    new_text+=augmented_text
                else:
                    new_text+=sents[k]
            newaugs.append([new_text,flag])
            #print('save augmentation for para changed sent {}'.format(idx))
    return newaugs

In [18]:
newaugs=[]
newaugs=run_augmentation(keyboard_aug,newaugs,df)
print(len(newaugs))


786


In [19]:

newaugs=run_augmentation(spelling_aug,newaugs,df)
print(len(newaugs))

1572


In [11]:
url='https://drive.google.com/file/d/1-AgeOgUe7srwFYqWWF56SMGaH8maPyOI/view?usp=sharing'
import pandas as pd

#url = 'https://drive.google.com/file/d/0B6GhBwm5vaB2ekdlZW5WZnppb28/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path)
newaugs=[]

In [12]:
df.head()

,Unnamed: 0,text,label
0,0,The 2016 presidential rac$ is undeniably the l...,False
1,1,Gragam briefly NeJtions the meteoric rise of f...,False
2,2,The resulting outcome is a world where those i...,False
3,3,"Due to tMese a?gor8tBms, fakw gews and conspir...",False
4,4,This part of the article reminded me of “ The ...,False


In [15]:
newaugs=run_augmentation(word2vec_aug,newaugs,df)
print(len(newaugs))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

In [16]:
print(len(newaugs))

118


In [17]:
#seldf=df[['Creation Content_last', 'Left_out_flag']]
newaugdf=pd.DataFrame(newaugs,columns=['text', 'label'])
outdf=pd.concat([df,newaugdf])

In [18]:
outdf.to_csv('word2vecaug_creations_flaged_aug.csv')

In [45]:
newaugs=run_augmentation(bert_sub_aug, newaugs,outdf)
print(len(newaugs))

608


In [29]:
%config Completer.use_jedi = False


In [38]:
import pandas as pd
outdf.drop('Unnamed: 0',axis=1,inplace=True)

In [43]:
outdf.shape

(218, 2)

In [ ]:
def save_em(name):
    newaugdf=pd.DataFrame(newaugs,columns=['text', 'label'])
    outdf1=outdf1.append(newaugdf,ignore_index=True)
    outdf1.to_csv(name)
    print('saved to {}'.format(name))

In [49]:
outdf1.to_csv('bert_aug_creationgs_flaged1.csv')

In [ ]:
newaugs=run_augmentation(xlnet_sub_aug, newaugs, outdf1)
print(len(newaugs))

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467M [00:00<?, ?B/s]

In [ ]:
save_em('xlnet_aug_creations_flagged.csv')

In [ ]:
'''
newaugs=[]
for idx,row in df.iterrows():
  text=row['Creation Content_last']
  sents=split_para(text)
  s=len(sents)
  #print('num of sent {}'.format(s))
  cnt=0
  #change one sentence each time.
  for i in range(s):
    aug = nac.KeyboardAug()
    #print('changed sentence {}'.format(i))
    augmented_text = aug.augment(sents[i])
    new_text=''
    for k in range(s):
      if k==i:
        new_text+=augmented_text
      else:
        new_text+=sents[k]
    newaugs.append(new_text)
    #print('save augmentation for para changed sent {}'.format(i))
  '''

In [2]:

%pprint off
newaugs

Pretty printing has been turned OFF


NameError: name 'newaugs' is not defined

In [1]:
newaugs[,1:3]

SyntaxError: invalid syntax (<ipython-input-1-699f6a5801f9>, line 1)